# Experiment 1

In [67]:
# build the vocab
!onmt_build_vocab -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure.yaml -n_sample 3500

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 10:16:22,308 INFO] Counter vocab from 3500 samples.
[2022-03-26 10:16:22,308 INFO] Build vocab on 3500 transformed examples/corpus.
[2022-03-26 10:16:23,154 INFO] Counters src:38
[2022-03-26 10:16:23,154 INFO] Counters tgt:39


In [1]:
# train the data
!onmt_train -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure.yaml

[2022-03-26 10:30:10,611 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-26 10:30:10,611 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 10:30:10,611 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-26 10:30:10,611 INFO] Parsed 2 corpora from -data.
[2022-03-26 10:30:10,611 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-26 10:30:10,611 INFO] Loading vocab from text file...
[2022-03-26 10:30:10,611 INFO] Loading src vocabulary from data/run/example.vocab.src
[2022-03-26 10:30:10,612 INFO] Loaded src vocab has 38 tokens.
[2022-03-26 10:30:10,612 INFO] Loading tgt vocabulary from data/run/example.vocab.tgt
[2022-03-26 10:30:10,612 INFO] Loaded tgt vocab has 39 tokens.
[2022-03-26 10:30:10,612 INFO] Building fields with vocab in counters...
[2022-03-26 10:30:10,613 INFO]  * tgt vocab size: 43.
[2022-03-26 10:30:10,613 INFO]  * src vocab size: 40.
[2022-03-2

In [69]:
# predict the test set
!onmt_translate -model data/run/model_step_16200.pt -src data/src_test.txt -output data/pred_experiment1.txt -verbose


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/modules/embeddings.py:195: UserWarning: Not merging with sum and positive feat_vec_size, but got non-default feat_vec_exponent. It will be unused.
  warnings.warn("Not merging with sum and positive "
[2022-03-26 10:26:42,448 INFO] Translating shard 0.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2022-03-26 10:26:42,622 INFO] 
SENT 1: ['@', 'd', 'm', 'I', 'n', 'I', 's', 't', '@', '

In [1]:
# read the prediction and test file
pred=open('/Users/shirleenyoung/Desktop/Groningen/NLP/data/pred_experiment1.txt', 'r').readlines()
test = open('/Users/shirleenyoung/Desktop/Groningen/NLP/data/tgt_test.txt', 'r').readlines()

In [2]:
# define a function to read the file and the line like 'b O n d' will become 'bOnd'
import re
def get_word(file):
    processed_lines =[]
    for word in file:
        word = re.sub(r'\n| ', '', word)
        processed_lines.append(word)
    return processed_lines

In [107]:
pred_lines = get_word(pred)
test_lines = get_word(test)
print(len(pred_lines), len(test_lines))

404 404


In [5]:
# read to merged english file
import pandas as pd
english = pd.read_csv('/Users/shirleenyoung/Desktop/Groningen/NLP/RevisitPinkerAndPrince/experiment_1/english_merged.txt', sep ='\t', names=['present_orth','past_orth','present_phon',	'past_phon', 'reg'])

In [6]:
# get the wrongly and correclty predicrted results and store them in two lists separately
def get_predictions(pred_lines, test_lines):
    correct_pred=[]
    wrong_pred = []
    for i in range(len(test_lines)):
        if test_lines[i]!= pred_lines[i]:
            wrong_pred.append({'past_phon': test_lines[i], 'predicted': pred_lines[i]})
        else:
            correct_pred.append(test_lines[i])
    return correct_pred, wrong_pred

In [7]:
correct_pred, wrong_pred = get_predictions(pred_lines, test_lines)

In [8]:
# wrongly predicted reuslts
part_df = pd.DataFrame(data = wrong_pred, columns = ['past_phon', 'predicted'])
part_df


,past_phon,predicted
0,drEdId,drOd
1,blaIndId,blaUnd
2,leI,laId
3,k&tIg@raIzd,kItIg@raIzd
4,b@g&n,b@gOn
5,s@spEndId,s@spEnt
6,Vnskru:d,Vnskru:
7,sO:,si:d
8,aUtr&n,aUtrVn
9,eIt,i:tId


In [22]:
print(len(correct_pred))
print(len(wrong_pred))
print('The accuracy of the test set is:', round(len(correct_pred)/404, 3))


384
20


0.9504950495049505

In [10]:
# fileter the wrongly predicted results from the english_merge.txt file
wrong_df = english.loc[english['past_phon'].isin([x['past_phon'] for x in wrong_pred])]

In [11]:
# check how many irregular words in the test set
english_irreg = english.loc[(english['past_phon'].isin(test_lines))&(english['reg']=='irreg')]
len(english_irreg)

13

In [12]:
# merge the wrongly predicted results with the engish_merged dataframe
wrong_df['pred'] = wrong_df['past_phon'].map(part_df.set_index('past_phon')['predicted'])

/opt/anaconda3/envs/nlp_project/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# have a closer look at the wrongly predicted irregular verbs
irreg = wrong_df.loc[wrong_df['reg']=='irreg']
irreg

,present_orth,past_orth,present_phon,past_phon,reg,pred
1511,begin,began,b@gIn,b@g&n,irreg,b@gOn
1631,forsake,forsook,fO:seIk,fO:sUk,irreg,fO:seIkt
2090,outrun,outran,aUtrVn,aUtr&n,irreg,aUtrVn
2494,sell,sold,sEl,s@Uld,irreg,sEld
2495,lie,lay,laI,leI,irreg,laId
2605,see,saw,si:,sO:,irreg,si:d
3149,foresee,foresaw,fO:si:,fO:sO:,irreg,fO:si:d
3236,sit,sat,sIt,s&t,irreg,sItId
3406,keep,kept,ki:p,kEpt,irreg,ki:pt
3958,eat,ate,i:t,eIt,irreg,i:tId


In [23]:
print('The accuracy of irregular verbs in the test set is: ', round(3/13, 3))

0.23076923076923078

In [14]:
# have a closer look at the wrongly predicted regular verbs
reg = wrong_df[wrong_df['reg']=='reg']
reg

,present_orth,past_orth,present_phon,past_phon,reg,pred
321,unscrew,unscrewed,Vnskru:,Vnskru:d,reg,Vnskru:
358,free,freed,fri:,fri:d,reg,fru:
1405,blind,blinded,blaInd,blaIndId,reg,blaUnd
1407,suspend,suspended,s@spEnd,s@spEndId,reg,s@spEnt
1550,hack,hacked,h&k,h&kt,reg,hVkt
2132,twang,twanged,tw&N,tw&Nd,reg,tw&N
2289,dread,dreaded,drEd,drEdId,reg,drOd
2367,categorize,categorized,k&tIg@raIz,k&tIg@raIzd,reg,kItIg@raIzd
2616,visualize,visualized,vIZjw@laIz,vIZjw@laIzd,reg,vIZj@laIzd
2964,freak,freaked,fri:k,fri:kt,reg,fr@Uk


## wug test

In [15]:
!onmt_translate -model data/run/model_step_16200.pt -src data/src.txt -output data/pred_experiment1_wug.txt -verbose


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/modules/embeddings.py:195: UserWarning: Not merging with sum and positive feat_vec_size, but got non-default feat_vec_exponent. It will be unused.
  warnings.warn("Not merging with sum and positive "
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/bin/onmt_translate", line 8, in <module>
    sys.exit(main())
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/bin/translate.py", line 54, in main
    translate(opt)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/bin/translate.py", line 26, in translate
    for i, (src_shard, tgt_shard, *features_shard) in enumerate(shard_pairs):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/utils/misc.py", line 35, in _split_corpus
    with open(path, "rb") as f:
FileNotFoundError: [Errno 2] No such file

In [142]:
wug_predictions = open('/Users/shirleenyoung/Desktop/Groningen/NLP/data/pred_experiment1_wug.txt', 'r').readlines()
wug_irr = open('/Users/shirleenyoung/Desktop/Groningen/NLP/RevisitPinkerAndPrince/experiment_1_wugs/tgt_irregular.txt', 'r').readlines()
wug_re = open('/Users/shirleenyoung/Desktop/Groningen/NLP/RevisitPinkerAndPrince/experiment_1_wugs/tgt_regular.txt', 'r').readlines()
wug_predictions = get_word(wug_predictions)
wug_irr = get_word(wug_irr)
wug_re = get_word(wug_re)

In [143]:
import pandas as pd
comp = pd.DataFrame(list(zip(wug_irr, wug_re, wug_predictions)), columns=['irregular', 'regular', 'prediction'])
comp

,irregular,regular,prediction


In [144]:
wug_irr_list =[]
wug_re_list =[]
for word in wug_predictions:
    if word in wug_irr:
        wug_irr_list.append(word)
    elif word in wug_re:
        wug_re_list.append(word)

In [145]:
print(len(wug_irr_list), len(wug_re_list))
print(wug_irr_list)

0 0
[]


# Experiment 2

In [116]:
!onmt_build_vocab -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure2.yaml -n_sample 15000


Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 13:29:07,668 INFO] Counter vocab from 15000 samples.
[2022-03-26 13:29:07,669 INFO] Build vocab on 15000 transformed examples/corpus.
[2022-03-26 13:29:10,425 INFO] Counters src:43
[2022-03-26 13:29:10,426 INFO] Counters tgt:38


In [117]:
!onmt_train -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure2.yaml

[2022-03-26 13:29:12,963 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-26 13:29:12,963 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 13:29:12,963 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-26 13:29:12,963 INFO] Parsed 2 corpora from -data.
[2022-03-26 13:29:12,964 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-26 13:29:12,964 INFO] Loading vocab from text file...
[2022-03-26 13:29:12,964 INFO] Loading src vocabulary from data2/run/example.vocab.src
[2022-03-26 13:29:12,964 INFO] Loaded src vocab has 43 tokens.
[2022-03-26 13:29:12,964 INFO] Loading tgt vocabulary from data2/run/example.vocab.tgt
[2022-03-26 13:29:12,964 INFO] Loaded tgt vocab has 38 tokens.
[2022-03-26 13:29:12,964 INFO] Building fields with vocab in counters...
[2022-03-26 13:29:12,965 INFO]  * tgt vocab size: 42.
[2022-03-26 13:29:12,965 INFO]  * src vocab size: 45.
[2022-03

In [118]:
!onmt_translate -model data2/run/model_step_75000.pt -src data2/src_test_tagged.txt -output data/pred_experiment2.txt -verbose


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/modules/embeddings.py:195: UserWarning: Not merging with sum and positive feat_vec_size, but got non-default feat_vec_exponent. It will be unused.
  warnings.warn("Not merging with sum and positive "
[2022-03-26 14:18:13,578 INFO] Translating shard 0.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2022-03-26 14:18:13,793 INFO] 
SENT 1: ['<V;PST>', '@', 'd', 'm', 'I', 'n', 'I', 's', 

In [119]:
test_lines = open('/Users/shirleenyoung/Desktop/Groningen/NLP/data/tgt_test.txt', 'r').readlines()
test_lines = get_word(test_lines)

In [120]:
pred2=open('/Users/shirleenyoung/Desktop/Groningen/NLP/data/pred_experiment2.txt', 'r').readlines()
pred_lines2 = get_word(pred2)

In [121]:
correct_pred2, wrong_pred2 = get_predictions(pred_lines2, test_lines)

In [122]:
part_df2 = pd.DataFrame(data = wrong_pred2, columns = ['past_phon', 'predicted'])
part_df2

,past_phon,predicted
0,@Uv@keIm,@Uv@kVmd
1,frOTt,frOTd
2,leI,laId
3,b@g&n,b@gInd
4,sO:,si:d
5,aUtr&n,aUtrVn
6,eIt,i:tId
7,kEpt,ki:pt
8,Vn3:Tt,Vn3:Td
9,fO:sUk,fO:seIkt


In [125]:
print(len(correct_pred2))
print(len(wrong_pred2))
390/404

390
14


0.9653465346534653

In [101]:
len(list(english['past_phon']))

4039

In [126]:
wrong_df2 = english.loc[english['past_phon'].isin([x['past_phon'] for x in wrong_pred2])]
wrong_df2

correct_df2 = english.loc[english['past_phon'].isin(correct_pred2)]
english.loc[(english['reg']=='irreg') & (english['past_phon'].isin(test_lines))]

correct_df2[correct_df2['reg'] =='irreg']

,present_orth,past_orth,present_phon,past_phon,reg
732,bid,bade,bId,beId,irreg
1254,dive,dived,daIv,daIvd,irreg
2494,sell,sold,sEl,s@Uld,irreg


In [127]:
wrong_df2['pred'] = wrong_df2['past_phon'].map(part_df2.set_index('past_phon')['predicted'])

/opt/anaconda3/envs/nlp_project/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [128]:
wrong_irreg2= wrong_df2[wrong_df2['reg']=='irreg']
wrong_irreg2

,present_orth,past_orth,present_phon,past_phon,reg,pred
283,overcome,overcame,@Uv@kVm,@Uv@keIm,irreg,@Uv@kVmd
1511,begin,began,b@gIn,b@g&n,irreg,b@gInd
1631,forsake,forsook,fO:seIk,fO:sUk,irreg,fO:seIkt
2090,outrun,outran,aUtrVn,aUtr&n,irreg,aUtrVn
2495,lie,lay,laI,leI,irreg,laId
2605,see,saw,si:,sO:,irreg,si:d
3149,foresee,foresaw,fO:si:,fO:sO:,irreg,fO:si:d
3236,sit,sat,sIt,s&t,irreg,sItId
3406,keep,kept,ki:p,kEpt,irreg,ki:pt
3958,eat,ate,i:t,eIt,irreg,i:tId


In [129]:
wrong_reg2= wrong_df2[wrong_df2['reg']=='reg']
wrong_reg2

,present_orth,past_orth,present_phon,past_phon,reg,pred
358,free,freed,fri:,fri:d,reg,frEd
1291,bead,beaded,bi:d,bi:dId,reg,bEdId
1477,froth,frothed,frOT,frOTt,reg,frOTd
3620,unearth,unearthed,Vn3:T,Vn3:Tt,reg,Vn3:Td


however, after the multitasking training, no irregular words were predicted correctly as well, but why there are only 6 irregular words.It is observed that the English_merge.txt in experiment 2 has different data with English_merge.txt in experiment1. **But why do they use same target set?**

# Challenge

## Learning the past tense in the form of phonological transcription

In [36]:
from makeWordListDutch import createDatasets

# use the phonological transcription as the training and test data
pronList = createDatasets('pron')
print("\nThere are", len(pronList), "phonetical wordpairs:")
print("   PRESENT:       PAST:       REGULAR:")
print(pronList[:6], "and so on...") # print 6 examples



There are 5796 phonetical wordpairs:
   PRESENT:       PAST:       REGULAR:
[('a m b I d @', 'a m b a d @', False), ('a m b I t', 'a m b A t', False), ('a n s x M w @', 'a n s x M w d @', True), ('a n s x M w', 'a n s x M w d @', True), ('a n s x M w t', 'a n s x M w d @', True), ('a n v a r d @', 'a n v a r d @', True)] and so on...


In [47]:
from sklearn.model_selection import train_test_split
# firstly get the irregular and regular verb list
dutch_phon_reg = [word for word in pronList if word[2]==True]
dutch_phon_irre = [word for word in pronList if word[2] != True]

# devide the tuple into the training and target data
dutch_phon_reg_src = [word[0] for word in dutch_phon_reg]
dutch_phon_reg_tgt = [word[1] for word in dutch_phon_reg]
dutch_phon_irre_src =[word[0] for word in dutch_phon_irre]
dutch_phon_irre_tgt = [word[1] for word in dutch_phon_irre]

# split the regular verb list and irregular verbs at 8:1:1 ratio
X_train_reg,X_test_reg, y_train_reg, y_test_reg = train_test_split(dutch_phon_reg_src, dutch_phon_reg_tgt,random_state =123, test_size = 0.2, train_size=0.8)
X_dev_reg, X_test_reg, y_dev_reg, y_test_reg = train_test_split(X_test_reg, y_test_reg, random_state =123,test_size = 0.5, train_size=0.5)

X_train_irreg,X_test_irreg, y_train_irreg, y_test_irreg = train_test_split(dutch_phon_irre_src, dutch_phon_irre_tgt,random_state =123, test_size = 0.2, train_size=0.8)
X_dev_irreg, X_test_irreg, y_dev_irreg, y_test_irreg = train_test_split(X_test_irreg, y_test_irreg, random_state =123,test_size = 0.5, train_size=0.5)


In [58]:
import codecs
# define a function to store the training and test data in the txt file as in the previous experiments
def save_file(lst, docname):
    with codecs.open (docname, 'wb', 'utf-8') as f:
        for word in lst:
            f.write(str(word) +'\n')


In [59]:

# write the train dataset to the txt file
X_train= X_train_reg+X_train_irreg
y_train = y_train_reg+ y_train_irreg

save_file(X_train, 'src_train_dutch.txt')
save_file(y_train, 'tgt_train_dutch.txt')

In [60]:
# same for the valid set
X_dev= X_dev_reg+X_dev_irreg
y_dev = y_dev_reg+ y_dev_irreg

save_file(X_dev, 'src_valid_dutch.txt')
save_file(y_dev, 'tgt_valid_dutch.txt')


In [61]:
# and the test set
X_test= X_test_reg+X_test_irreg
y_test = y_test_reg+ y_test_irreg

save_file(X_test, 'src_test_dutch.txt')
save_file(y_test, 'tgt_test_dutch.txt')


In [62]:
!onmt_build_vocab -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure3.yaml -n_sample 5000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 11:45:42,549 INFO] Counter vocab from 5000 samples.
[2022-03-26 11:45:42,549 INFO] Build vocab on 5000 transformed examples/corpus.
[2022-03-26 11:45:43,209 INFO] Counters src:39
[2022-03-26 11:45:43,210 INFO] Counters tgt:39


In [63]:
!onmt_train -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure3.yaml

[2022-03-26 11:46:16,137 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-26 11:46:16,137 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 11:46:16,137 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-26 11:46:16,137 INFO] Parsed 2 corpora from -data.
[2022-03-26 11:46:16,137 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-26 11:46:16,138 INFO] Loading vocab from text file...
[2022-03-26 11:46:16,138 INFO] Loading src vocabulary from data_dutch/example.vocab.src
[2022-03-26 11:46:16,138 INFO] Loaded src vocab has 39 tokens.
[2022-03-26 11:46:16,138 INFO] Loading tgt vocabulary from data_dutch/example.vocab.tgt
[2022-03-26 11:46:16,139 INFO] Loaded tgt vocab has 39 tokens.
[2022-03-26 11:46:16,139 INFO] Building fields with vocab in counters...
[2022-03-26 11:46:16,139 INFO]  * tgt vocab size: 43.
[2022-03-26 11:46:16,139 INFO]  * src vocab size: 41.
[2022-

In [64]:
!onmt_translate -model data_dutch/model_step_23185.pt -src data_dutch/src_test_dutch.txt -output data_dutch/pred_experiment_dutch.txt -verbose

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/modules/embeddings.py:195: UserWarning: Not merging with sum and positive feat_vec_size, but got non-default feat_vec_exponent. It will be unused.
  warnings.warn("Not merging with sum and positive "
[2022-03-26 12:06:32,981 INFO] Translating shard 0.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2022-03-26 12:06:33,173 INFO] 
SENT 1: ['b', '@', 'n', 'a', 'd', 'r', '}', 'k']
PRED 1

In [65]:
pred_dutch = open('/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/pred_experiment_dutch.txt', 'r').readlines()

In [130]:
pred_dutch_processed = []
for word in pred_dutch:
    word = re.sub(r'\n', '', word)
    pred_dutch_processed.append(word)

In [138]:
correct_pred_dutch, wrong_pred_dutch = get_predictions(pred_dutch_processed, y_test)
print(len(wrong_pred_dutch), len(correct_pred_dutch))

# get the correct predictions for the irregular verbs
correct_pred_dirreg = []
for word in correct_pred_dutch:
    if word in y_test_irreg:
        correct_pred_dirreg.append(word)

wrong_pred_dirreg = []
for word in wrong_pred_dutch:
    if word['past_phon']in y_test_irreg:
        wrong_pred_dirreg.append(word)

print('The accuracy for the Dutch past tense prediction overall in phonological transcription is:', round(len(correct_pred_dirreg)/(len(correct_pred_dirreg)+len(wrong_pred_dirreg)),3))

print('The accuracy for the Dutch past tense prediction overall in phonological transcription is:', round(560/580, 3))

20 560
The accuracy for the Dutch past tense prediction overall in phonological transcription is: 0.889
The accuracy for the Dutch past tense prediction overall in phonological transcription is: 0.966


## Learning of the past tense in the form of orthographic form

In [146]:
orthoList = createDatasets('ortho')


There are 5918 orthographical wordpairs:
   PRESENT:     PAST:    REGULAR:
[('aanbidden', 'aanbaden', False), ('aanbid', 'aanbad', False), ('aanbidt', 'aanbad', False), ('aanschouwen', 'aanschouwden', True), ('aanschouw', 'aanschouwde', True), ('aanschouwt', 'aanschouwde', True)] and so on...


In [164]:
dutch_orth_reg = [word[1] for word in orthoList if word[2]==True]
dutch_orth_irreg = [word[1] for word in orthoList if word[2] != True]

In [169]:
dutch_orth_reg_src = [' '.join(list(word)) for word in dutch_orth_reg]
dutch_orth_reg_tgt = [' '.join(list(word)) for word in dutch_orth_reg]
dutch_orth_irre_src =[' '.join(list(word)) for word in dutch_orth_irreg]
dutch_orth_irre_tgt = [' '.join(list(word)) for word in dutch_orth_irreg]

In [170]:
X_train_orth_reg, X_test_orth_reg, y_train_orth_reg, y_test_orth_reg = train_test_split(dutch_orth_reg_src, dutch_orth_reg_tgt,random_state =123, test_size = 0.2, train_size=0.8)
X_test_orth_reg, X_dev_orth_reg, y_test_orth_reg, y_dev_orth_reg = train_test_split(X_test_orth_reg, y_test_orth_reg, train_size=0.5, test_size = 0.5, random_state =123)

X_train_orth_irreg, X_test_orth_irreg, y_train_orth_irreg, y_test_orth_irreg = train_test_split(dutch_orth_irre_src, dutch_orth_irre_tgt,random_state =123, test_size = 0.2, train_size=0.8)
X_test_orth_irreg, X_dev_orth_irreg, y_test_orth_irreg, y_dev_orth_irreg = train_test_split(X_test_orth_irreg, y_test_orth_irreg, train_size=0.5, test_size = 0.5, random_state =123)

In [173]:
print(y_train_orth_irreg[:10])

['v e r z o n d', 's c h r e e d', 'w r o n g', 'h i e l p', 'm i s p r e e s', 'o n t s t a k', 'h e r w o n n e n', 'v e r z o n d', 'h i e f', 'n e e g']


In [176]:
X_train_orth = X_train_orth_reg + X_train_orth_irreg
X_dev_orth = X_dev_orth_reg + X_dev_orth_irreg
X_test_orth = X_test_orth_reg + X_test_orth_irreg

y_train_orth = y_train_orth_reg+y_train_orth_irreg
y_dev_orth = y_dev_orth_reg+y_dev_orth_irreg
y_test_orth = y_test_orth_reg+y_test_orth_irreg


save_file(X_train_orth, '/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/dutch_orth/src_train_dutch.txt')
save_file(X_dev_orth, '/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/dutch_orth/src_valid_dutch.txt')
save_file(X_test_orth, '/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/dutch_orth/src_test_dutch.txt')
save_file(y_train_orth, '/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/dutch_orth/tgt_train_dutch.txt')
save_file(y_dev_orth, '/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/dutch_orth/tgt_valid_dutch.txt')
save_file(y_test_orth, '/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/dutch_orth/tgt_test_dutch.txt')

In [177]:
!onmt_build_vocab -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure4.yaml -n_sample 5000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 20:06:41,469 INFO] Counter vocab from 5000 samples.
[2022-03-26 20:06:41,469 INFO] Build vocab on 5000 transformed examples/corpus.
[2022-03-26 20:06:42,197 INFO] Counters src:26
[2022-03-26 20:06:42,197 INFO] Counters tgt:26


In [178]:
!onmt_train -config /Users/shirleenyoung/Desktop/Groningen/NLP/data_structure4.yaml

[2022-03-26 20:07:22,234 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-26 20:07:22,234 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-26 20:07:22,234 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-26 20:07:22,234 INFO] Parsed 2 corpora from -data.
[2022-03-26 20:07:22,234 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-26 20:07:22,234 INFO] Loading vocab from text file...
[2022-03-26 20:07:22,234 INFO] Loading src vocabulary from data_dutch/dutch_orth/example.vocab.src
[2022-03-26 20:07:22,235 INFO] Loaded src vocab has 26 tokens.
[2022-03-26 20:07:22,235 INFO] Loading tgt vocabulary from data_dutch/dutch_orth/example.vocab.tgt
[2022-03-26 20:07:22,236 INFO] Loaded tgt vocab has 26 tokens.
[2022-03-26 20:07:22,236 INFO] Building fields with vocab in counters...
[2022-03-26 20:07:22,236 INFO]  * tgt vocab size: 30.
[2022-03-26 20:07:22,236 INFO]  * src 

In [179]:
!onmt_translate -model data_dutch/dutch_orth/model_step_23185.pt -src data_dutch/dutch_orth/src_test_dutch.txt -output data_dutch/dutch_orth/pred_experiment_dutch_orth.txt -verbose

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/onmt/modules/embeddings.py:195: UserWarning: Not merging with sum and positive feat_vec_size, but got non-default feat_vec_exponent. It will be unused.
  warnings.warn("Not merging with sum and positive "
[2022-03-26 20:27:23,502 INFO] Translating shard 0.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2022-03-26 20:27:23,678 INFO] 
SENT 1: ['b', 'e', 'o', 'o', 'r', 'd', 'e', 'e', 'l', '

In [180]:
pred_dutch_orth = open('/Users/shirleenyoung/Desktop/Groningen/NLP/data_dutch/dutch_orth/pred_experiment_dutch_orth.txt', 'r').readlines()

In [185]:
pred_dutch_processed_orth = []
for word in pred_dutch_orth:
    word = re.sub(r'\n', '', word)
    pred_dutch_processed_orth.append(word)

print(pred_dutch_processed_orth[:10])

['b e o o r d e e l d e n', 's n o e i d e', 'm i s l e i d d e', 'b e v o o r r e c h t t e', 'p e l d e', 'k n e u s d e', 'v e r k n o e i d e', 'b e e f d e', 'p i e k t e', 's c h i t t e r d e n']


In [189]:
correct_pred_dutch_orth, wrong_pred_dutch_orth = get_predictions(pred_dutch_processed_orth, y_test_orth)
print(len(wrong_pred_dutch_orth), len(correct_pred_dutch_orth))

print('The accuracy for the Dutch past tense prediction overall in phonological transcription is:', round(591/591, 3))

0 591
The accuracy for the Dutch past tense prediction overall in phonological transcription is: 1.0
